In [ ]:
from selenium import webdriver
from datetime import datetime
from selenium.webdriver.common.by import By
import pandas as pd
import time
import json
from bs4 import BeautifulSoup

In [ ]:
def scrape_all_events():
  """
  Scrape all events from Parimatch for a given sport.

  Args:
    sport: The sport to scrape data for.

  Returns:
    A DataFrame containing the scraped data.
  """

  # Get the URL for the events page.
  url = f"https://parimatch-in.com/en/cricket/"

  # Scrape the page.
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html.parser")

  # Get the events.
  events = soup.find_all("div", class_="event-container")

  # Create a DataFrame to store the data.
  df = pd.DataFrame()

  # Add the events to the DataFrame.
  for event in events:
    event_name = event.find("h3").text
    event_url = event.find("a").get("href")
    df = df.append({"Event Name": event_name, "Event URL": event_url}, ignore_index=True)

  # Return the DataFrame.
  return df

In [ ]:
df1 = scrape_parimatch_all()

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://parimatch.com/en/sports/cricket/"

response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")

matches = soup.find_all("div", class_="match-item")

for match in matches:

    match_name = match.find("h3").text
    match_date = match.find("span", class_="match-date").text
    match_time = match.find("span", class_="match-time").text
    team1 = match.find("span", class_="team1").text
    team2 = match.find("span", class_="team2").text
    odds = match.find("span", class_="odds").text

    print(f"{match_name} | {match_date} | {match_time} | {team1} | {team2} | {odds}")


In [ ]:
print(df)

In [ ]:
browser = webdriver.Firefox()
browser.get('https://gg.bet/en/live')

In [ ]:
browser.get('https://gg.bet/en/live')
html=browser.page_source
time.sleep(3)
table = browser.find_element(By.CLASS_NAME, "overviewRow__container___2uPYc")

In [ ]:
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
for a in soup:
    print(a)

In [ ]:
a=table.get_attribute('outerHTML')
BeautifulSoup.prettify(a)

In [ ]:
def scrape_surebets():
    browser = webdriver.Firefox()
    browser.get('https://en.surebet.com/surebets')
    html=browser.page_source
    time.sleep(10)
    table = browser.find_element(By.CSS_SELECTOR, "#surebets-table")
    table_html = table.get_attribute('outerHTML')
    df = pd.read_html(table_html)[0]
    df.drop(columns=["▼Profit","Unnamed: 7","Unnamed: 8"])
    browser.close()
    return df

In [ ]:
df=scrape_surebets()

In [ ]:
df

In [ ]:
df.to_csv('/media/adi/032E-9FF0/projects/web bets/surebets.csv', index=False)

In [ ]:
a=soup.find("class=__app-LogoTitle-name logoTitle__name___3_ywM")

In [ ]:
print(a)

In [ ]:
df.drop(columns=["▼Profit","Unnamed: 7","Unnamed: 8"]).to_records

In [ ]:
df

In [ ]:
"meta"

In [ ]:
df.to_records()

In [49]:
import requests
from bs4 import BeautifulSoup

url = "https://parimatch.com/en/sports/cricket/"

response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")

matches = soup.find_all("div", class_="match-item")

for match in matches:

    match_name = match.find("h3").text
    match_date = match.find("span", class_="match-date").text
    match_time = match.find("span", class_="match-time").text
    team1 = match.find("span", class_="team1").text
    team2 = match.find("span", class_="team2").text
    odds = match.find("span", class_="odds").text

    print(f"{match_name} | {match_date} | {match_time} | {team1} | {team2} | {odds}")


In [111]:
import requests
from bs4 import BeautifulSoup

# Specify the URL of the page you want to scrape
url = 'https://parimatch-in.com/en/cricket/'

# Make a request to the URL
response = requests.get(url)

# Create a BeautifulSoup object from the response
soup = BeautifulSoup(response.content, 'html.parser')

# Find the elements that contain the data you want to scrape
elements = soup.find_all('div')

# Loop over the elements and extract the data
for element in elements:
    # Get the name of the event
    event_name = element.find('h3')

    # Get the time of the event
    event_time = element.find('span', class_='time')

    # Get the odds for the event
    odds = element.find_all('span', class_='odds')

    # Print the data
    print(event_name, event_time, odds)


None None []
None None []
None None []
None None []
None None []
None None []


In [138]:
elements = soup.find_all('div')


In [144]:
import requests
from bs4 import BeautifulSoup

# Specify the URL of the page you want to scrape
url = 'https://parimatch.com/en/'

# Make a request to the URL
response = requests.get(url)

# Create a BeautifulSoup object from the response
soup = BeautifulSoup(response.content, 'html.parser')

# Find the elements that contain the data you want to scrape
elements = soup.find_all('div', class_='event-card')
import requests
from bs4 import BeautifulSoup

# Specify the URL of the page you want to scrape
url = 'https://parimatch.com/en/'

# Make a request to the URL
response = requests.get(url)

# Create a BeautifulSoup object from the response
soup = BeautifulSoup(response.content, 'html.parser')

# Find the elements that contain the data you want to scrape
elements = soup.find_all('div', class_='event-card')

# Loop over the elements and extract the data
for element in elements:
    # Get the name of the event
    event_name = element.find('h3').text

    # Get the time of the event
    event_time = element.find('span', class_='time').text

    # Get the odds for the event
    odds = element.find_all('span', class_='odds')

    # Print the data
    print(event_name, event_time, odds)

    # Save the data to a file
    with open('parimatch_data.csv', 'w') as f:
        f.write('event_name,event_time,odds\n')
        for event_name, event_time, odds in zip(event_names, event_times, odds):
            f.write('{event_name},{event_time},{odds}\n'.format(event_name=event_name, event_time=event_time, odds=', '.join(odds)))
(elements)
# Loop over the elements and extract the data
for element in elements:
    # Get the name of the event
    event_name = element.find('h3').text

    # Get the time of the event
    event_time = element.find('span', class_='time').text

    # Get the odds for the event
    odds = element.find_all('span', class_='odds')

    # Print the data
    print(event_name, event_time, odds)

    # Save the data to a file
    with open('parimatch_data.csv', 'w') as f:
        f.write('event_name,event_time,odds\n')
        for event_name, event_time, odds in zip(event_names, event_times, odds):
            f.write('{event_name},{event_time},{odds}\n'.format(event_name=event_name, event_time=event_time, odds=', '.join(odds)))


NameError: name 'pri1nt' is not defined